In [2]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.corpus import wordnet
import enchant
from nltk.metrics import edit_distance

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import LinearSVC


import requests

# Loading data yelp api

In [66]:
API_key = 'ZPdOK1WUGHAXLl9TBhGD9GG8p7wjE6TelZtakUyDLbpS0qI92EpxiAuA5OYmdINFO_EcDG4pnqjN_5-BkPGhCOXNUxebMOTFvgTU9XcUEZ4wd7wzDKj35tKqK6LbXXYx'
Endpoint = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization' : 'bearer {}'.format(API_key)}
Parameters = {'term':'restaurant', 'limit':50 }
data = list()

for offset in range(0, 3350, 50):
    Parameters.update({'offset': offset})
    response = requests.get(url = Endpoint, params = Parameters, headers = headers)
    data.append(response.json())

In [67]:
len(data)

67

In [18]:
#headers = {'Authorization' : 'bearer {}'.format(API_key)}
Parameters = {}
reviews = list()
count = 0
for subset50 in data:
    for biz in subset50['businesses']:
        Endpoint = 'https://api.yelp.com/v3/businesses/' + biz['id'] +'/reviews'
        response = requests.get(url = Endpoint, params = Parameters, headers = headers)
        reviews.append(response.json())
    count = count + 1
    print("loaded ", count*3*50, 'reviews')

loaded  150 reviews
loaded  300 reviews
loaded  450 reviews
loaded  600 reviews
loaded  750 reviews
loaded  900 reviews
loaded  1050 reviews
loaded  1200 reviews
loaded  1350 reviews
loaded  1500 reviews
loaded  1650 reviews
loaded  1800 reviews
loaded  1950 reviews
loaded  2100 reviews
loaded  2250 reviews
loaded  2400 reviews
loaded  2550 reviews
loaded  2700 reviews
loaded  2850 reviews
loaded  3000 reviews


KeyError: 'businesses'

In [24]:
reviews_list = list()
for reviews3 in reviews:
    for review in reviews3['reviews']:
#        (review_id, stars, text) = review['id'], review['rating'], review['text']
        id_review_text_time = review['id'], review['rating'], review['text'], review['time_created']        
        reviews_list.append(id_review_text_time)

In [25]:
df = pd.DataFrame(reviews_list, columns = ['review_id' , 'stars', 'text' , 'date'])

In [27]:
df.shape

(3000, 4)

In [28]:
df['review_year'] = df['date'].apply(lambda x:int(x[:4]))

In [29]:
df.head(3)

,review_id,stars,text,date,review_year
0,7pktjPusbXX1g55uEZqJmw,5,This was the perfect start to my Sunday mornin...,2019-11-26 17:16:27,2019
1,vR-BfefUsGAza6MlqEJc5Q,5,A good friend of mine gifted me the Clinton St...,2019-11-22 09:01:54,2019
2,W9w6LOUA__Z_LJkrefGSHw,3,I am not sure what the hype is with Clinton St...,2019-11-21 20:43:59,2019


# Cleaning data

## Delete '\r\n', '\n\n', '\n', '\r'

In [35]:
# (Perkins) Python 3 Text Processing with NLTK 3 Cookbook
class Newline_Replacer(object):
    def replace(self, text):
        s = text
        s = s.replace('\r\n', ' ')
        s = s.replace('\n\n', ' ') 
        s = s.replace('\n', ' ')
        s = s.replace('\r', ' ') 
        return s

## Delete Extra spaces 

In [36]:
# (Perkins) Python 3 Text Processing with NLTK 3 Cookbook
class Extra_Spaces_Replacer(object):
    def replace(self, text):
        s = text
        s = re.sub('\s\s+', ' ', s)
        return s

## Delete Word reduction

In [38]:
# (Perkins) Python 3 Text Processing with NLTK 3 Cookbook
rep_patterns = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'can not'),
    (r'i\'m', 'i am'),
    (r'I\'m', 'I am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are'),
    (r'(\w+)\'d', '\g<1> would'),
]

In [39]:
# (Perkins) Python 3 Text Processing with NLTK 3 Cookbook
class Word_Reduction_Replacer(object):
    def __init__(self, patterns=rep_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    
    def replace(self, text):
        s = text

        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)

        return s

## Delete repeating letters

In [40]:
# (Perkins) Python 3 Text Processing with NLTK 3 Cookbook 
class Repeat_Replacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'

    def replace(self, word):
        if wordnet.synsets(word):
            return word
        
        repl_word = self.repeat_regexp.sub(self.repl, word)

        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word

## Spelling Correction

In [41]:
# (Perkins) Python 3 Text Processing with NLTK 3 Cookbook

class Spelling_Replacer(object):
    def __init__(self, dict_name='en', max_dist=1):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = max_dist
    
    def replace(self, word):
        if self.spell_dict.check(word):
            return word
    
        suggestions = self.spell_dict.suggest(word)
    
        if suggestions and edit_distance(word, suggestions[0]) <= self.max_dist:
            return suggestions[0]
        else:
            return word

In [42]:
newline_replacer = Newline_Replacer()
extra_spaces_replacer = Extra_Spaces_Replacer()
word_reduction_replacer = Word_Reduction_Replacer()
repeat_replacer = Repeat_Replacer()
spell_replacer = Spelling_Replacer()

## Apply cleaning functions

In [43]:
df['text'] = df['text'].apply(newline_replacer.replace)

In [44]:
df['text'] = df['text'].apply(extra_spaces_replacer.replace)

In [45]:
df['text'] = df['text'].apply(word_reduction_replacer.replace)

In [46]:
df['text'] = df['text'].apply(repeat_replacer.replace)

In [47]:
df['text'] = df['text'].apply(spell_replacer.replace)

In [49]:
df.head(4)

,review_id,stars,text,date,review_year
0,7pktjPusbXX1g55uEZqJmw,5,This was the perfect start to my Sunday mornin...,2019-11-26 17:16:27,2019
1,vR-BfefUsGAza6MlqEJc5Q,5,A god friend of mine gifted me the Clinton Str...,2019-11-22 09:01:54,2019
2,W9w6LOUA__Z_LJkrefGSHw,3,I am not sure what the hype is with Clinton St...,2019-11-21 20:43:59,2019
3,XIho0jlcBxQUsiYCVYkqbg,5,nice fod! We got a nice table. They have so ma...,2019-11-25 05:43:25,2019


## Destribution of reviews by year

In [50]:
df.groupby(['review_year'])['review_id'].size().reset_index()

,review_year,review_id
0,2012,3
1,2017,1
2,2018,12
3,2019,2984


## Distribution of reviews by star

In [51]:
df.groupby(['stars'])['review_id'].size().reset_index()


,stars,review_id
0,1,127
1,2,180
2,3,393
3,4,875
4,5,1425


## Sentiment flag

If stars >= 4, sentiment = 1
If starts < 4, sentiment = 0

In [52]:
df['sentiment'] = df['stars'].apply(lambda x: 1 if x >= 4 else 0)

In [53]:
df.head(3)

,review_id,stars,text,date,review_year,sentiment
0,7pktjPusbXX1g55uEZqJmw,5,This was the perfect start to my Sunday mornin...,2019-11-26 17:16:27,2019,1
1,vR-BfefUsGAza6MlqEJc5Q,5,A god friend of mine gifted me the Clinton Str...,2019-11-22 09:01:54,2019,1
2,W9w6LOUA__Z_LJkrefGSHw,3,I am not sure what the hype is with Clinton St...,2019-11-21 20:43:59,2019,0


## Removing stop words

In [54]:
english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

df['cleaned_text'] = df['text'].apply(lambda x: remove_stop_words(x))

## Lemmatization

In [55]:
nltk.download('wordnet')

def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

df['cleaned_text'] = df['cleaned_text'].apply(lambda x: get_lemmatized_text(x))

[nltk_data] Downloading package wordnet to /home/ilya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Create Train/Validation/Test Split

In [57]:
X_train_set, X_nottrain, y_train, y_nottrain = train_test_split(df['text'], 
                                                    df['sentiment'], 
                                                    test_size=0.30, 
                                                    random_state=42)

X_test_set, X_val_set, y_test, y_val = train_test_split(X_nottrain, 
                                                    y_nottrain, 
                                                    test_size=0.5, 
                                                    random_state=42)

## CountVectorizer

In [58]:
cv = CountVectorizer(ngram_range=(1,2))
cv.fit(X_train_set)
X_train = cv.transform(X_train_set)
X_test = cv.transform(X_test_set)
X_val = cv.transform(X_val_set)

In [59]:
X_train.shape

(2100, 34302)

In [60]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(solver = 'lbfgs', 
                            penalty = 'l2', 
                            C=c,
                           max_iter= 500)
    lr.fit(X_train, y_train)
    print ("For C = %s, Accuracy: %s, F1-score: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val)), f1_score(y_val, lr.predict(X_val))))

For C = 0.01, Accuracy: 0.7377777777777778, F1-score: 0.8479381443298969
For C = 0.05, Accuracy: 0.7666666666666667, F1-score: 0.8612945838837517
For C = 0.25, Accuracy: 0.7666666666666667, F1-score: 0.857916102841678
For C = 0.5, Accuracy: 0.7688888888888888, F1-score: 0.8586956521739131
For C = 1, Accuracy: 0.7688888888888888, F1-score: 0.8586956521739131


In [61]:
final_model = LogisticRegression(solver = 'lbfgs', 
                            penalty = 'l2', 
                            C=0.25,
                           max_iter= 500)
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s, F1-score: %s" 
           % (accuracy_score(y_test, final_model.predict(X_test)), f1_score(y_test, final_model.predict(X_test))))

Final Accuracy: 0.7911111111111111, F1-score: 0.877284595300261


In [62]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
print("Best Positive")
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:15]:
    print (best_positive)

print("Best Negative")

for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:15]:
    print (best_negative)

Best Positive
('delicious', 0.647324913390588)
('excelent', 0.6296718845487629)
('amazing', 0.6010545525994486)
('spot', 0.5934995661428131)
('the best', 0.510448554136048)
('loved', 0.49278528175454245)
('cozy', 0.45028331077467987)
('perfect', 0.43812461583068724)
('dishes', 0.43006198205893337)
('best', 0.4294134104402576)
('glad', 0.41625494914674843)
('was great', 0.40272593243449223)
('find', 0.40270864956673325)
('go', 0.4009883537404277)
('pork', 0.38712647507835346)
Best Negative
('ok', -0.6792476913284964)
('reviews', -0.636617266136754)
('who', -0.6324898407365795)
('terible', -0.580288699807557)
('disapointed', -0.5677040595298375)
('other', -0.5606252967175549)
('this place', -0.5562274347483906)
('service', -0.5523607029324997)
('few', -0.546669382832302)
('average', -0.517830734353114)
('like', -0.4435788051535116)
('excited', -0.4391506524854406)
('bad', -0.4270176639740535)
('however the', -0.4145313852911916)
('not', -0.40941320096011613)


## Baseline SVM

In [64]:
for c in [0.001, 0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c, max_iter=500)
    svm.fit(X_train, y_train)
    print ("For C = %s, Accuracy: %s, F1-score: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val)), f1_score(y_val, svm.predict(X_val))))

For C = 0.001, Accuracy: 0.7377777777777778, F1-score: 0.8479381443298969
For C = 0.01, Accuracy: 0.7711111111111111, F1-score: 0.8617449664429531
For C = 0.05, Accuracy: 0.7644444444444445, F1-score: 0.8559782608695652
For C = 0.25, Accuracy: 0.7622222222222222, F1-score: 0.8532235939643347
For C = 0.5, Accuracy: 0.7622222222222222, F1-score: 0.8532235939643347
For C = 1, Accuracy: 0.7622222222222222, F1-score: 0.8532235939643347


In [65]:
final_model = LinearSVC(C=0.01, max_iter=500)

final_model.fit(X_train, y_train)
print ("Final Accuracy: %s, F1-score: %s" 
           % (accuracy_score(y_test, final_model.predict(X_test)), f1_score(y_test, final_model.predict(X_test))))

Final Accuracy: 0.7977777777777778, F1-score: 0.882882882882883
